# Installing Necessary Libraries

In [ ]:
!pip3.10 install -q langchain
!pip3.10 install -q chromadb
!pip3.10 install -q langchain-google-genai
!pip3.10 install -q google-generativeai
!pip3.10 install -q unstructured
!pip3.10 install -q -U langchain-community

In [ ]:
!pip3.10 install -q trycourier

In [ ]:
import textwrap
import pandas as pd
from pprint import pprint
from courier.client import Courier

import google.generativeai as genai

from IPython.display import Markdown

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [41]:
import os
os.environ["GOOGLE_API_KEY"] = "" # Enter your Gemini API Key
os.environ["Courier_API_KEY"] = "" # Enter your Courier Production API Key 

In [ ]:
genai.configure(api_key = os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel('gemini-pro')

# Summary

In [ ]:
f = open("sample.txt", "r")
content = f.read()

In [ ]:
response = model.generate_content(f"""

{content}

---

Please create a detailed Minutes of the Meeting based on above content. It must include following titles:
1. Title
2. Meeting Date
3. Meeting Time
4. Meeting Location
5. Attendees
6. Agenda
7. Actions Taken
8. Adjournment
9. Meeting adjourned at

Ensure clarity and conciseness in the minutes. Use bullet points or numbered lists where appropriate.""")

to_markdown(response.text)

# Email

In [ ]:
club = pd.read_csv("") # Enter path to csv file containing employee records - Name, Email, Role

In [ ]:
# names = ["Alice Johnson", "Bob Smith", "Carol Davis", "David Wilson", "Eva Brown", "Frank White", "Grace Lee",
#          "Henry Harris", "Ivy Martin","Jack Thompson"]

# email_ids = ["alice.johnson@example.com", "bob.smith@example.com", "carol.davis@example.com", "david.wilson@example.com",
#              "eva.brown@example.com", "frank.white@example.com", "grace.lee@example.com", "henry.harris@example.com",
#              "ivy.martin@example.com","jack.thompson@example.com"]

# roles = ["Junior Core", "Senior Core", "Board Member", "Junior Core", "Senior Core", "Board Member", "Junior Core",
#          "Senior Core", "Board Member", "Junior Core"]

# club = pd.DataFrame([names, email_ids, roles]).T

In [ ]:
club = club.set_axis(['Name', 'Email', 'Roles'], axis = 1)

In [ ]:
prompt_roles = club["Roles"].unique().tolist()

In [ ]:
prompt_names = club["Name"].unique().tolist()

In [ ]:
response = model.generate_content(f"""

{content}

---

Based on the above content, please create a 2d Python list named 'future_meetings' that stores data about all future meetings discussed in the transcript. Each
list contains two elements:

1. **Meeting Details:**
   - Title
   - Date (Follow the Format Example for Date: Tuesday, June 11, 2024)
   - Time
   - Location (if applicable)

2. **Participants**
 - Must be either Everyone, a certain group {prompt_roles} or individuals {prompt_names}.

 Just create future meetings explicitly discussed in the above given transcript. Don't try to fabricate any meetings.""")

to_markdown(response.text)


In [ ]:
future_meetings = []
exec(response.text[10:-3])
pprint(future_meetings)

In [28]:
meetings = []
for i in future_meetings:
    response = model.generate_content(f"""Consider given details:

    {i[0]}

    ------

    Now, based on above content, create a short and brief email message.
    Use proper salutation, communicate the message succinctly and kindly end with a Thank You.
    Do not include any sign-off or name at the end.""")

    emails = []
    participants = i[1]

    if len(participants) == 1:
        if participants[0].lower() == "everyone":
            emails.extend(club["Email"].tolist())

        elif any(participants[0].lower() in j.lower() for j in prompt_roles):
            emails.extend(club[club["Roles"] == participants[0].title()]["Email"].tolist())

    else:
        for person in participants:
            if any(person.lower() in j.lower() for j in prompt_roles):
                emails.extend(club[club["Roles"] == person.title()]["Email"].tolist())

            else:
                name = [s for s in club["Name"].tolist() if person.lower() in s.lower()]
                emails.extend(club[club["Name"] == name[0]]["Email"].tolist())

    emails = list(set(emails))
    meeting = [response.text, emails]
    meetings.append(meeting)

In [ ]:
for count, meeting in enumerate(meetings):
    print("\n\n" + str(count + 1) + ". " + meeting[0])
    print("\nSend to: " + ", ".join(meeting[1]))

In [38]:
def send_email(emails, ans, auth):
    client = Courier(authorization_token = auth)
    for email in emails:
        client.send(message = {"to": {"email": f"{email}"},
                                       "content": {"title": f"Meeting Info",
                                                   "body": f"""{ans}"""},
                                       "data": {"note": f"\nDo not reply back to this email. \n\n {ans}"},
                                       "routing": {"method": "single", "channels": ["email"]}})
    return True

In [43]:
for meeting in meetings:
    send_email(emails = meeting[1], ans = meeting[0], auth = os.environ["Courier_API_KEY"])

# Q/A

In [ ]:
query_text = "On the agenda, who had the first item?"

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores.chroma import Chroma
import getpass
import shutil

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

CHROMA_PATH = "chroma" # Change this everytime you run this cell again
DATA_PATH = "data"


def main():
    generate_data_store()


def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.txt")
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100,
                                                   length_function = len, add_start_index = True)
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[1]
    print(document.page_content)
    print(document.metadata)

    return chunks

def save_to_chroma(chunks: list[Document]):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    db = Chroma.from_documents(chunks, embeddings, persist_directory = CHROMA_PATH)
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


if __name__ == "__main__":
    main()

In [ ]:
from dataclasses import dataclass
from langchain.vectorstores.chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAI

CHROMA_PATH = "chroma" # Change this to above created path

PROMPT_TEMPLATE = """
Please provide a precise and thorough answer to the question below, relying exclusively on the given context.
Ensure your response is accurate, complete, and clearly articulated. If the context does not contain enough information,
please indicate this explicitly. Also, give the answer in complete sentences.

Context:
{context}

---

Question:
{question}

Answer the question based only on the above context:
"""

db = Chroma(persist_directory = CHROMA_PATH, embedding_function = embeddings)

results = db.similarity_search_with_relevance_scores(query_text, k = 3)

if len(results) == 0 or results[0][1] < 0.6:
    print(f"Unable to find matching results.")

context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context = context_text, question = query_text)

response_text = model.generate_content(prompt)

to_markdown(response_text.text)